# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [ ]:
# pip install gmaps

In [34]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [35]:
file = "cities_pd.csv"

cities_pd = pd.read_csv(file)

cities_pd

,city,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,codrington,-38.2667,141.9667,285.87,55,8,1.50,AU,1659932146
1,rikitea,-23.1203,-134.9692,294.44,77,100,16.51,PF,1659932146
2,port alfred,-33.5906,26.8910,287.14,99,88,0.45,ZA,1659932112
3,union de san antonio,21.1000,-101.9667,289.85,68,91,1.65,MX,1659932148
4,butaritari,3.0707,172.7902,300.76,75,92,5.43,KI,1659932148
...,...,...,...,...,...,...,...,...,...
550,manggar,-2.8833,108.2667,302.43,67,4,5.85,ID,1659933135
551,kolobovo,56.7020,41.3417,291.64,90,99,2.62,RU,1659933136
552,kochki,54.3358,80.4806,293.34,64,57,3.73,RU,1659933137
553,gangapur,26.4833,76.7167,304.33,59,75,2.99,IN,1659933138


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [31]:
gmaps.configure(api_key=g_key)

locations = cities_pd[['Lat' , 'Lng']]

humidity = cities_pd['Humidity'].astype(float)

# locations 

# humidity

In [32]:
fig = gmaps.figure(display_toolbar=True, display_errors=True, zoom_level=2, tilt=45, center=(0, 0), layout=None, map_type='ROADMAP', mouse_handling='COOPERATIVE')

heat_layer = gmaps.heatmap_layer(locations, weights=None, max_intensity=50, dissipating=False, point_radius=1, opacity=0.6, gradient=None)

fig.add_layer(heat_layer)

fig

# heat_layer

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows with null values.

In [33]:
cities_weather_complete = pd.DataFrame(cities_pd.dropna(how='any'))

# cities_weather_complete

len(cities_weather_complete)
# len(cities_pd)

553

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [ ]:
# Add marker layer ontop of heat map


# Display figure
